In [ ]:
%pylab nbagg
#%matplotlib inline
#%matplotlib widget

In [ ]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt

import scipy.io as io
import h5py

import phate
from sklearn.cluster import KMeans
import os

from functions.load_data import  go_avalanches
from functions.plot import  plot_figure_2D, plot_figure_2D_patient

# DATASET EXTRACTION

In [ ]:
#remove suject 11,15,20
subjects = ['43','39','38','35','34','29','26','21','20','19','18','17','15','13','9','8','6','5']
Nsubs=44
subject='1'
nregions=90

In [ ]:
#original sampling at 1024 Hz. Data downsampled to 256 Hz.
f_MEG=1024
Tlen=160*f_MEG

nedges=int(nregions**2/2-nregions/2)

iTriup= np.triu_indices(nregions,k=1)  #nedges=len(iTriup[0])

In [ ]:
#Edge to Nodes
dic={}
for e in range(nedges):
    dic['%d'%e]=[iTriup[0][e],iTriup[1][e]]

In [ ]:
path_data = os.path.dirname(os.path.realpath(__file__))+'/data/'
f = h5py.File(path_data+'serie_Melbourne.mat','r')

In [ ]:
struArray = f['D']
data={}
for i in range(Nsubs):
    data['%d'%i]=np.swapaxes(f[struArray[i, 0]][:nregions,:],0,1)
Avalanches_human=go_avalanches(data[subject],thre=3,direc=0,binsize=1)

In [ ]:
avalanches_bin = []
avalanches_sum = []
for subject in subjects:
    Avalanches_human=go_avalanches(data[subject],thre=3,direc=0,binsize=1)
    Mag=np.mean(Avalanches_human['Zbin'],axis=1)
    out=[[] for i in range(len(Avalanches_human['ranges']))]
    out_sum=[[] for i in range(len(Avalanches_human['ranges']))]
    for kk1 in range(len(Avalanches_human['ranges'])):
        begin = Avalanches_human['ranges'][kk1][0]
        end = Avalanches_human['ranges'][kk1][1]
        sum_kk = np.sum(Avalanches_human['Zbin'][begin:end,:],0)
        out_sum[kk1] = sum_kk
        out[kk1]= np.zeros(nregions)
        out[kk1][np.where(sum_kk>=1)]=1

    avalanches_bin.append(np.concatenate([out],axis=1))
    avalanches_sum.append(np.concatenate([out_sum],axis=1))
    out = np.concatenate([out],axis=1)
    out_sum = np.concatenate([out_sum],axis=1)


In [ ]:
# # save one subject
# io.savemat('subject_'+str(subject)+'.mat',{'source_reconstruction_MEG':data,
#                                            'avalanches_binarize':out,
#                                            'avalanches_sum':out_sum})

# Shuffle data

In [ ]:
## example shuffle data
shuffle_index= np.arange(out.shape[0])
np.random.shuffle(shuffle_index)
out_shuffle = np.squeeze(out[shuffle_index])
phate_operator = phate.PHATE(n_components=3, n_jobs=-2, decay=1.0,  n_pca=10, gamma=1, knn=5, knn_dist='cosine'   )
Y_phate = phate_operator.fit_transform(out)
Y_phate_shuffle = phate_operator.fit_transform(out_shuffle)

# Visualisation of the output shuffle

fig = plt.figure()
plt.scatter(Y_phate[:,0],Y_phate[:,1])
fig = plt.figure()
plt.scatter(Y_phate_shuffle[:,0],Y_phate_shuffle[:,1])


# all data

In [ ]:
knn_dist = 'cosine'
mds_dist = 'cosine'
knn_dist_name=knn_dist
for n_components in range(2,6):
    for n_pca in [5]:
        for gamma in [-1.0,0.0,1.0]:
            if not os.path.exists("../projection_data/all_subject_Y_phate_knn_dist_"+knn_dist_name
                                  +"_mds_dist_"+mds_dist+"_nb_comp_"+str(n_components)+"_nb_pca_"+str(n_pca)
                                  +"_gamma_"+str(gamma)+".npy"):
                phate_operator = phate.PHATE(n_components=n_components, n_jobs=-2, decay=1.0,  n_pca=n_pca,
                                             gamma=gamma, knn=5, knn_dist=knn_dist, mds_dist=mds_dist )
                Y_phate = phate_operator.fit_transform(avalanches_bin)
                np.save("all_subject_Y_phate_knn_dist_"+knn_dist_name+"_mds_dist_"+mds_dist
                        +"_nb_comp_"+str(n_components)+"_nb_pca_"+str(n_pca)+"_gamma_"+str(gamma)+".npy",Y_phate)

In [ ]:
for knn_dist_name,mds_dist,n_components,n_pca,gamma,nb_cluster in [('cosine','cosine',2,5,1.0,10),
                                                                   ('cosine','cosine',2,5,0.0,10),
                                                                   ('cosine','cosine',2,5,-1.0,10)]:
    file = "../projection_data/all_subject_Y_phate_knn_dist_"+knn_dist_name+"_mds_dist_"+mds_dist\
           +"_nb_comp_"+str(n_components)+"_nb_pca_"+str(n_pca)+"_gamma_"+str(gamma)
    data = np.load(file+'.npy')
    plot_figure_2D(data,file,KMeans(n_clusters=nb_cluster, random_state=123).fit_predict(data))

knn_dist = 'cosine'
mds_dist = 'cosine'
knn_dist_name=knn_dist
nb_cluster= 5
for n_components in range(2,6):
    for n_pca in [5]:
        for gamma in [-1.0,0.0,1.0]:
            file = "all_subject_Y_phate_knn_dist_"+knn_dist_name+"_mds_dist_"+mds_dist\
                   +"_nb_comp_"+str(n_components)+"_nb_pca_"+str(n_pca)+"_gamma_"+str(gamma)+".npy"
            data=np.load(file)
            plot_figure_2D(data,file,KMeans(n_clusters=nb_cluster, random_state=123).fit_predict(data))
            plot_figure_2D_patient(data,file,avalanches_sum)

# Cluster Result

In [ ]:
cluster = KMeans(n_clusters=nb_cluster, random_state=123).fit_predict(data)
file = "../projection_data/all_subject_Y_phate_knn_dist_cosine_mds_dist_cosine_nb_comp_2_nb_pca_5_gamma_-1.0.npy"
save = True
plot = True

In [ ]:
cluster_patient_data = []
begin = 0
for avalanche in avalanches_bin:
    end = begin + len(avalanche)
    cluster_patient_data.append(cluster[begin:end])
    begin = end
cluster_patient = np.empty((len(avalanches_bin),nb_cluster))
transition = np.empty((len(subjects),nb_cluster,nb_cluster))
histograms_patient = np.empty((len(subjects),nb_cluster))
for index_patient, cluster_k in enumerate(cluster_patient_data):
    hist = np.histogram(cluster_k,bins=nb_cluster,range=(0,12))
    histograms_patient[index_patient,:]=hist[0]
    next_step = cluster_k[1:]
    step = cluster_k[:-1]
    for i in range(nb_cluster):
        data = next_step[np.where(step==i)]
        percentage_trans = np.bincount(data)/len(data)
        if len(percentage_trans) < nb_cluster:
            percentage_trans = np.concatenate([percentage_trans,np.zeros(nb_cluster-percentage_trans.shape[0])])
        transition[index_patient,i,:] = percentage_trans
if plot:
    for index_patient, cluster_k in enumerate(cluster_patient_data):
        fig, axs = plt.subplots(1,2)
        axs[0].hist(cluster_k,bins=nb_cluster,range=(0,12))
        im = axs[1].imshow(transition[index_patient],vmin = 0.0)
        fig.colorbar(im)
if save :
    data = np.load(file)
    io.savemat('../projection_data/cluster_18_patients.mat',
               {'avalanches_binarize':np.concatenate(avalanches_bin),
                'cluster_index':KMeans(n_clusters=nb_cluster, random_state=123).fit_predict(data),
                'PHATE_position':data,
                'transition_matrix':transition,
                'histogram':histograms_patient})